# AXI UART usecase

## Block Diagram

![Figure 1. Block Diagram](img/block.png)

## Pinout

PMOD A: 1:RX, 2:TX. baud 9600

## Import Libraries

In [1]:
from pynq import MMIO
from pynq import Overlay

import sys
sys.path.insert(1, './src')
from uartlite import *

## Download bitstream

In [2]:
ol = Overlay("./bits/uartlitetest.bit")
ol.download()

## Initialize AXI UART

In [3]:
# Address of the ip core
ADDRESS = 0x42c00000  
uart = UartAXI(ADDRESS)

# Setup AXI UART register
uart.setupCtrlReg()

In [4]:
# Loopback test
uart.write('heldsklfjs\n')
print(uart.readLine())

heldsklfjs

